In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
from torch.nn import CrossEntropyLoss
from torch.optim import Adam


In [2]:
class CustomLogoDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        for label, subdir in enumerate(['Fake', 'Genuine']):
            folder_path = os.path.join(self.root_dir, subdir)
            for filename in os.listdir(folder_path):
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    self.images.append(os.path.join(folder_path, filename))
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Image transformations
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [3]:
# Prepare datasets and dataloaders
train_dataset = CustomLogoDataset('FakeReal Logo Detection dataset/train', transform=image_transforms)
test_dataset = CustomLogoDataset('FakeReal Logo Detection dataset/test', transform=image_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
# Model setup
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)  # Adjust for desired EfficientNet version
optimizer = Adam(model.parameters(), lr=0.001)
criterion = CrossEntropyLoss()


Loaded pretrained weights for efficientnet-b0


In [5]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader)}")

Epoch 1/10, Loss: 0.24273083651704447
Epoch 2/10, Loss: 0.05935526786128148
Epoch 3/10, Loss: 0.034938442055135965
Epoch 4/10, Loss: 0.02863863198962506
Epoch 5/10, Loss: 0.030564694948095297
Epoch 6/10, Loss: 0.04584065236572531
Epoch 7/10, Loss: 0.014763503240482393
Epoch 8/10, Loss: 0.02101607100866247
Epoch 9/10, Loss: 0.030131693186593197
Epoch 10/10, Loss: 0.010362703724586893


In [6]:
# Evaluation
model.eval()
correct = total = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total}%')

Accuracy of the network on the test images: 90.3030303030303%


In [7]:
# Save the model
torch.save(model.state_dict(), 'efficient_net_logo_detection_model_second.pth')